In [ ]:
%load_ext autoreload
%autoreload 2

https://colab.research.google.com/drive/1uCuA6We9T5r0WljspEHWPHXCT_2bMKUy

In [ ]:
import mediapipe as mp
import math
import pathlib
import matplotlib.pyplot as plt

import cv2 as cv

from climbing_wire.utils.data import get_package_fol


In [ ]:
import mediapipe.python.solutions.pose as mp_pose
import mediapipe.python.solutions.drawing_utils as mp_drawing
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles


In [ ]:
img_fol = get_package_fol("sample_square")


In [ ]:
# Read images with OpenCV.
imgs = {img_path.name: cv.imread(str(img_path)) for img_path in img_fol.iterdir()}
img_names = list(imgs)


In [ ]:
from climbing_wire.landmark.compute import compute_landmarks
from climbing_wire.utils.cv import cv_imshow

img1 = imgs[img_names[0]]

pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.5,
    model_complexity=2,
)

pose_landmarks = compute_landmarks(img1, pose)

annotated_image = img1.copy()
mp_drawing.draw_landmarks(
    annotated_image,
    pose_landmarks,
    mp_pose.POSE_CONNECTIONS,
    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
)

cv_imshow(annotated_image)
